# Testing Mantid's CylindricalAbsorption wrapper
<div class="alert alert-block alert-info">
<p><b>Information:</b> This notebook shows how to use <code>scipp</code>'s implementation of <code>Mantid</code>'s <code>CylinderAbsorption</code> algorithm.</p>

<p><b>Requirements:</b> To run this notebook, you need <code>scippneutron</code> installed as well as the Python script <code>absorption.py</code> placed in the same folder as this notebook.</p>
        
The test file is available from Mantid: <a href="http://198.74.56.37/ftp/external-data/MD5/d5ae38871d0a09a28ae01f85d969de1e">PG3_4844_event.nxs</a>. Once downloaded the file should be renamed and placed in the same folder.
</div>

In [ ]:
import numpy as np
import scipp as sc
import scippneutron as scn

from absorption import absorption_correction

filename = "PG3_4844_event.nxs"

## Loading data 

In [ ]:
dataset = scn.load(filename=filename)
dataset

The file is loaded as a `DataArray` with following sizes:

In [ ]:
print(f"Dataset's dimensions: {dataset.dims} and shape: {dataset.shape}.")

In [ ]:
dataset_lambda = scn.convert(dataset, 'tof', 'wavelength', scatter=True)
dataset_lambda

In [ ]:
print((f"The dataset to be corrected from absorption has dimensions:" 
       f"{dataset_lambda.dims} and shape: {dataset_lambda.shape}."))

## Defining absorption parameters

Our absorption should be calculated for the sample.

In [ ]:
scatter_from = "Sample"

## Calculating absorption

Now we are ready to calculate the correction, passing the sample definition and example cylinder dimensions to the calculator.
We need these, since the data file contains no sample definition.
This calculation takes some time.

In [ ]:
correction = absorption_correction(filename,
                                   ScatterFrom=scatter_from,
                                   CylinderSampleHeight=3,
                                   CylinderSampleRadius=10)

<!-- The correction dataset uses wavelengths units, so we need to convert it back to time-of-flight, as in the original sample. -->
The sample dataset is converted to wavelengths units before normalizing 

The absorption correction has been calculated to match the input dataset over bins ranging from

In [ ]:
correction.coords['spectrum'].values[0]

to

In [ ]:
correction.coords['spectrum'].values[-1]

## More complex parameter definition

We may also define other parameters required for Mantid's algorithm.

In [ ]:
sample_number_density = 0.07192
number_wavelength_points = 5
cylinder_sample_height = 4
cylinder_sample_radius = 0.4
number_of_slices = 2
attenuation_x_section = 5.08
scattering_x_section = 5.1

# We use the minimum and maximum values of the wavelength's coordinates of the sample in order to calculate the absorption
min_lambda = np.min(dataset_lambda.bins.constituents['data'].coords['wavelength'].values)
max_lambda = np.max(dataset_lambda.bins.constituents['data'].coords['wavelength'].values)

These parameters can just be appended to the `absorption_correction` call.

In [ ]:
correction_with_params = absorption_correction(filename,
                                   lambda_binning=(min_lambda, max_lambda, 2000),
                                   ScatterFrom=scatter_from,
                                   AttenuationXSection=attenuation_x_section,
                                   ScatteringXSection=scattering_x_section,
                                   SampleNumberDensity=sample_number_density,
                                   NumberOfWavelengthPoints=number_wavelength_points,
                                   CylinderSampleHeight=cylinder_sample_height,
                                   CylinderSampleRadius=cylinder_sample_radius,
                                   NumberOfSlices=number_of_slices)

In [ ]:
correction

In [ ]:
print((f"Correction dataset's spectrum first and last values:"
       f"{correction_with_params.coords['spectrum'].values[0]}, "
       f"{correction_with_params.coords['spectrum'].values[-1]}."))

Finally, we can apply the correction to the initial data array.

In [ ]:
dataset_lambda

In [ ]:
correction_with_params

In [ ]:
del correction_with_params.coords['source_position']
del correction_with_params.coords['sample_position']
del correction_with_params.coords['position']

In [ ]:
np.max(correction_with_params.values)

In [ ]:
corrected = dataset_lambda.bins / sc.lookup(func=correction_with_params, dim='wavelength')

In [ ]:
corrected

In [ ]:
sc.plot(corrected, title='Data after absorption corrrection')

In [ ]:
sc.plot(dataset_lambda, title='Data before absorption corrrection')

### 1D plot

In [ ]:
lambda_bins =  sc.Variable(
    dims=['wavelength'],
    values=np.linspace(min_lambda, max_lambda, 2000),
    unit=sc.units.angstrom)

histo_ini = sc.histogram(dataset_lambda, bins=lambda_bins)
histo_abs_corr = sc.histogram(corrected, bins=lambda_bins)

In [ ]:
# select spectrum to plot
spectr_nb = 1100

sc.plot({'Ini': histo_ini['spectrum', spectr_nb].values, 'AbsCorr': histo_abs_corr['spectrum', spectr_nb].values}, 
        grid=True,
        labels={'axis-0': 'wavelength (Angstrom)'},
        title=f'Comparison of initial and absorption corrected spectrum {spectr_nb}')